In [17]:
import requests
import json
import numpy as np
import pandas as pd
import datetime as dt
from shroomdk import ShroomDK


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Setting

In [4]:
#API Setting
API_KEY = '33552129-3743-4219-8d00-ebe02f43878b'
sdk = ShroomDK(API_KEY )

In [32]:
# SQL Scripts
# CryptoPunks Sales
cp_sales = """
WITH NFT_SALES AS (
	SELECT 
  		BLOCK_TIMESTAMP,
    	TX_HASH,
    	EVENT_TYPE,
    	PROJECT_NAME,
    	TOKENID,
    	PRICE_USD,
  		SELLER_ADDRESS,
  		BUYER_ADDRESS
	FROM
  		ethereum.core.ez_nft_sales
  	WHERE PRICE > 0
  		AND PRICE_USD > 0
  		AND NFT_ADDRESS IN ('0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb') -- CryptoPunks		
),

  AGG_TABLE AS (
    SELECT
    	DATE_TRUNC('day',BLOCK_TIMESTAMP) AS DATE,
      	COUNT(TX_HASH) AS SALES_NUMBER,
      	AVG(SALES_NUMBER) OVER(ORDER BY DATE ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS MA30_SALES, --30DAYS MOVING AVR.
      	SUM(SALES_NUMBER) OVER(ORDER BY DATE) AS TOTAL_RUNNING_SALES,
      	SUM(PRICE_USD) AS USD_VOL,
      	AVG(USD_VOL) OVER (ORDER BY DATE ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS MA30_USD,  --30DAYS MOVING AVR.
      	SUM(USD_VOL) OVER(ORDER BY 1) AS VOL_ACCUMULATED,
      	MEDIAN(PRICE_USD) AS MEDIAN_PRICE,
      	MIN(PRICE_USD) AS FLOOR_PRICE
    FROM
    	NFT_SALES
    GROUP BY 1
    ORDER BY 1 ASC)

SELECT * FROM  AGG_TABLE
"""

# Top 10 CryptoPunks Traders
cp_top10 = """

--Buyers
WITH NFT_PURCHASES AS (
	SELECT 
  		BUYER_ADDRESS,
    	COUNT(TX_HASH) AS PURCHASES,
    	SUM(PRICE_USD) AS PURCHASE_USD
  		
	FROM
  		ethereum.core.ez_nft_sales
  	WHERE PRICE > 0
  		AND PRICE_USD > 0
  		AND NFT_ADDRESS = '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb' -- CryptoPunks
  	GROUP BY 1
),

  NFT_SELLS AS (
	SELECT 
  		SELLER_ADDRESS,
    	COUNT(TX_HASH) AS SELLS,
    	SUM(PRICE_USD) AS SELL_USD
  		
	FROM
  		ethereum.core.ez_nft_sales
  	WHERE PRICE > 0
  		AND PRICE_USD > 0
  		AND NFT_ADDRESS = '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb' -- CryptoPunks	
  	GROUP BY 1
),

NFT_BUY_SELL AS (
	SELECT
  		
		*
  	FROM
  		NFT_PURCHASES
  		FULL OUTER JOIN NFT_SELLS ON BUYER_ADDRESS = SELLER_ADDRESS
	--ORDER BY TRADES_COUNT DESC
  )
  
SELECT
 		BUYER_ADDRESS,
  		IFNULL(PURCHASES,0) AS PURCHASES,
  		IFNULL(PURCHASE_USD,0) AS PURCHASE_USD,
  		RANK() OVER(ORDER BY PURCHASE_USD DESC) AS TOP_BUYERS_USD,
  		IFNULL(SELLS,0) AS SELLS,
  		IFNULL(SELL_USD,0) AS SELL_USD,
  		RANK() OVER(ORDER BY SELL_USD DESC) AS TOP_SELLERS_USD,
  		IFNULL(PURCHASE_USD,0) + IFNULL(SELL_USD,0) AS  VOL_USD,
  		RANK() OVER(ORDER BY VOL_USD DESC) AS TOP_TRADERS_USD,
  		IFNULL(PURCHASES,0) +IFNULL(SELLS,0) AS TRADES_COUNT,
  		RANK() OVER(ORDER BY TRADES_COUNT DESC) AS TOP_TRADERS
  		
FROM 
  NFT_BUY_SELL
  --WHERE BUYER_ADDRESS = '0xfd845e07717b0329d3f19fc920c97fba0bc4ee31' OR SELLER_ADDRESS = '0x24e267d13314dd742d86c3321e727b801ed99542'
ORDER BY PURCHASE_USD DESC
"""

# CryptoPunks Sales - Whales
whales = """
WITH NFT_BUYERS AS (
	SELECT
  		BUYER_ADDRESS,
  		SUM(PRICE_USD) AS PURCHASES_USD
  	FROM
  		ethereum.core.ez_nft_sales
  	GROUP BY 1
),
  
  NFT_SALES AS (
	SELECT 
  		BLOCK_TIMESTAMP,
    	TX_HASH,
    	EVENT_TYPE,
    	PROJECT_NAME,
    	TOKENID,
    	PRICE_USD,
  		SELLER_ADDRESS,
  		BUYER_ADDRESS,
  		(CASE
  		WHEN BUYER_ADDRESS IN (SELECT BUYER_ADDRESS FROM NFT_BUYERS WHERE  PURCHASES_USD >= 500000) THEN 'Whale' -- Explain Whale definition
  		ELSE 'Typical User'
  	END) AS BUYER_TYPE,
  	(CASE
  		WHEN SELLER_ADDRESS IN (SELECT BUYER_ADDRESS FROM NFT_BUYERS WHERE  PURCHASES_USD >= 500000) THEN 'Whale' -- Explain Whale definition
  		ELSE 'Typical User'
  	END) AS SELLER_TYPE
	FROM
  		ethereum.core.ez_nft_sales
  	WHERE PRICE > 0
  		AND PRICE_USD > 0
  		AND NFT_ADDRESS = '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb' -- CryptoPunks
),

  AGG_TABLE AS (
    SELECT
    	DATE_TRUNC('day',BLOCK_TIMESTAMP) AS DATE,
   		BUYER_TYPE,
      	COUNT(TX_HASH) AS SALES_NUMBER,
      	SUM(PRICE_USD) AS USD_VOL 	
    FROM
    	NFT_SALES
    GROUP BY 1,2
    ORDER BY 1 ASC)

SELECT * FROM AGG_TABLE
"""

# CryptoPunks Sales - P/L & Holding Time
pl_holdtime = """
WITH NFT_BUYERS AS (
	SELECT
  		BUYER_ADDRESS,
  		SUM(PRICE_USD) AS PURCHASES_USD
  	FROM
  		ethereum.core.ez_nft_sales
  	GROUP BY 1
),
  
  NFT_SALES AS (
	SELECT 
  		BLOCK_TIMESTAMP,
    	TX_HASH,
    	EVENT_TYPE,
    	PROJECT_NAME,
    	TOKENID,
    	PRICE_USD,
  		SELLER_ADDRESS,
  		BUYER_ADDRESS,
  		(CASE
  		WHEN BUYER_ADDRESS IN (SELECT BUYER_ADDRESS FROM NFT_BUYERS WHERE  PURCHASES_USD >= 500000) THEN 'Whale' -- Explain Whale definition
  		ELSE 'Typical User'
  	END) AS BUYER_TYPE,
  	(CASE
  		WHEN SELLER_ADDRESS IN (SELECT BUYER_ADDRESS FROM NFT_BUYERS WHERE  PURCHASES_USD >= 500000) THEN 'Whale' -- Explain Whale definition
  		ELSE 'Typical User'
  	END) AS SELLER_TYPE
	FROM
  		ethereum.core.ez_nft_sales
  	WHERE PRICE > 0
  		AND PRICE_USD > 0
  		AND NFT_ADDRESS = '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb' -- CryptoPunks
),

BUY_SEL AS (
    SELECT
  		S.BLOCK_TIMESTAMP AS SELL_DATE,
        B.BLOCK_TIMESTAMP AS BUY_DATE,
        B.BUYER_ADDRESS,
        S.SELLER_ADDRESS,
        B.TOKENID,
        B.BUYER_TYPE,
        S.SELLER_TYPE,
        S.PRICE_USD - B.PRICE_USD as PL_USD,
        (S.PRICE_USD - B.PRICE_USD)/B.PRICE_USD as PL_PERCENT,
        DATEDIFF('day',BUY_DATE,SELL_DATE) AS HOLDING_PERIOD
  
  	FROM  NFT_SALES B
  	INNER JOIN NFT_SALES S ON B.BUYER_ADDRESS = S.SELLER_ADDRESS
  	WHERE B.TOKENID = S.TOKENID
        AND SELL_DATE >= BUY_DATE
  	ORDER BY 1)

SELECT * FROM BUY_SEL

"""

## Data Frames

### CryptoPunks Sales

In [18]:
query_result = sdk.query(cp_sales)
data = query_result.records
list_data = []
for row in data:
    list_data.append(row)
df = pd.DataFrame(data =list_data)
df.date = df.date.apply(lambda x: dt.datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))
display(df)

,date,sales_number,ma30_sales,total_running_sales,usd_vol,ma30_usd,vol_accumulated,median_price,floor_price
0,2020-05-05,3,3.000,3,4376.13,4376.130000,2.582681e+09,1255.250,1028.80
1,2020-05-07,3,3.000,6,798.03,2587.080000,2.582681e+09,275.940,201.72
2,2020-05-08,6,4.000,12,2296.39,2490.183333,2.582681e+09,297.530,116.82
3,2020-05-09,15,6.750,27,5129.63,3150.045000,2.582681e+09,266.250,124.80
4,2020-05-10,17,8.800,44,8345.29,4189.094000,2.582681e+09,378.930,126.38
...,...,...,...,...,...,...,...,...,...
788,2022-08-27,2,4.600,14633,262173.21,750525.028000,2.582681e+09,131086.605,100831.54
789,2022-08-28,1,4.533,14634,103972.81,737594.187333,2.582681e+09,103972.810,103972.81
790,2022-08-29,1,4.433,14635,153808.39,724935.941667,2.582681e+09,153808.390,153808.39
791,2022-08-30,1,4.366,14636,102787.89,703825.771333,2.582681e+09,102787.890,102787.89


In [19]:
df.to_excel('CryptoPunks Sales.xlsx', index =False )

### Top 10 CryptoPunks Traders

In [22]:
query_result = sdk.query(cp_top10)
data = query_result.records
list_data = []
for row in data:
    list_data.append(row)
df = pd.DataFrame(data =list_data)
display(df)

,buyer_address,purchases,purchase_usd,top_buyers_usd,sells,sell_usd,top_sellers_usd,vol_usd,top_traders_usd,trades_count,top_traders
0,0xef764bac8a438e7e498c2e5fccf0f174c3e3f8db,17,5.359424e+08,1141,5,590228.51,3293,5.365327e+08,1,22,198
1,0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,679,1.314855e+08,1142,406,82248623.83,2675,2.137342e+08,3,1085,1
2,0x69c488bcda156379b6661f08a35db627e5d467dd,13,2.610106e+07,1143,0,0.00,1,2.610106e+07,12,13,356
3,0x0e9aed5c7721c642a032812c2c4816f7d6cb87d7,133,2.009618e+07,1144,130,26898421.06,2679,4.699460e+07,5,263,4
4,0xfe2f279d3679bac2d07cf46c93503410ef9ca448,78,1.546365e+07,1145,71,18360630.93,2684,3.382428e+07,7,149,12
...,...,...,...,...,...,...,...,...,...,...,...
6189,None,0,0.000000e+00,1,1,70669.94,5422,7.066994e+04,4859,1,3214
6190,None,0,0.000000e+00,1,1,561429.85,3337,5.614298e+05,1409,1,3214
6191,None,0,0.000000e+00,1,1,33272.93,5860,3.327293e+04,5635,1,3214
6192,None,0,0.000000e+00,1,1,537766.99,3362,5.377670e+05,1467,1,3214


In [24]:
df.to_excel('Top 10 CryptoPunks Traders.xlsx', index =False )

### CryptoPunks Sales - Whales

In [26]:
query_result = sdk.query(whales)
data = query_result.records
list_data = []
for row in data:
    list_data.append(row)
df = pd.DataFrame(data =list_data)
df.date = df.date.apply(lambda x: dt.datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))
display(df)

,date,buyer_type,sales_number,usd_vol
0,2020-05-05,Whale,2,2284.05
1,2020-05-05,Typical User,1,2092.08
2,2020-05-07,Whale,2,522.09
3,2020-05-07,Typical User,1,275.94
4,2020-05-08,Typical User,5,1924.69
...,...,...,...,...
1390,2022-08-27,Whale,1,161341.67
1391,2022-08-28,Typical User,1,103972.81
1392,2022-08-29,Typical User,1,153808.39
1393,2022-08-30,Typical User,1,102787.89


In [27]:
df.to_excel('CryptoPunks Sales - Whales.xlsx', index =False )

### CryptoPunks Sales - P/L & Holding Time

In [33]:
query_result = sdk.query(pl_holdtime)
data = query_result.records
list_data = []
for row in data:
    list_data.append(row)
df = pd.DataFrame(data =list_data)
df.sell_date = df.sell_date.apply(lambda x: dt.datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))
display(df)

,sell_date,buy_date,buyer_address,seller_address,tokenid,buyer_type,seller_type,pl_usd,pl_percent,holding_period
0,2020-05-20,2020-05-14 11:09:56.000,0x51787a2c56d710c68140bdadefd3a98bff96feb4,0x51787a2c56d710c68140bdadefd3a98bff96feb4,4923,Whale,Whale,91.74,0.452702,6
1,2020-05-22,2020-05-09 22:42:36.000,0xcc81b4631c4b321421267c1d4b43e7b2d7c52a23,0xcc81b4631c4b321421267c1d4b43e7b2d7c52a23,6654,Typical User,Typical User,36.55,0.292869,13
2,2020-05-22,2020-05-20 22:31:07.000,0x63a9dbce75413036b2b778e670aabd4493aaf9f3,0x63a9dbce75413036b2b778e670aabd4493aaf9f3,8946,Whale,Whale,100.65,0.341893,2
3,2020-05-22,2020-05-14 11:08:03.000,0x51787a2c56d710c68140bdadefd3a98bff96feb4,0x51787a2c56d710c68140bdadefd3a98bff96feb4,4505,Whale,Whale,208.85,1.030595,8
4,2020-05-22,2020-05-14 19:15:38.000,0x63a9dbce75413036b2b778e670aabd4493aaf9f3,0x63a9dbce75413036b2b778e670aabd4493aaf9f3,3771,Whale,Whale,74.70,0.570011,8
...,...,...,...,...,...,...,...,...,...,...
6360,2022-08-21,2022-08-15 21:08:22.000,0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,8457,Whale,Whale,-20099.53,-0.148595,6
6361,2022-08-24,2021-12-27 19:45:37.000,0x0e94af7a715274345bccf9996f98f145d3ca1499,0x0e94af7a715274345bccf9996f98f145d3ca1499,5857,Whale,Whale,-170289.44,-0.603879,240
6362,2022-08-26,2021-09-25 05:25:42.000,0xf6651185a37378aad75e5cd1e3141bdd762b74d2,0xf6651185a37378aad75e5cd1e3141bdd762b74d2,2976,Typical User,Typical User,-283083.13,-0.716789,335
6363,2022-08-27,2021-07-31 17:08:13.000,0x0e9aed5c7721c642a032812c2c4816f7d6cb87d7,0x0e9aed5c7721c642a032812c2c4816f7d6cb87d7,4234,Whale,Whale,16967.86,0.117527,392


In [35]:
df.to_excel('CryptoPunks Sales-PL & Holding Time.xlsx', index =False )